In [1]:
#Load txt file with information about the gage sites
import csv
gageSite = open("160921_GageSites_LA.txt","r")
csvReader = csv.reader(gageSite, delimiter=' ')

In [2]:
siteList = []

for line in csvReader:
    if line[-1]=="LA":
        siteID = line[5]
        place = line[-2][:-1]
        pl = line[-3]
        if pl != "near" and pl != "at" and pl != "nr" and pl != "of" and pl != "NR" and pl != "AT":
            place = ' '.join([pl, place])
        siteList.append([siteID,place])

In [3]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#the following dataframe will be used later to identify the parish in which the gage is found
gages = pd.DataFrame(siteList,columns=['ID','City'])
gages.head()

,ID,City
0,02489500,Bogalusa
1,02491500,Franklinton
2,02492000,Bush
3,02492600,Pearl River
4,07344450,Greenwood


In [5]:
#Parsing the file with the daily gage height for all sites in Louisiana (LA)
gageMeanHeight = []
#gageHeight = open("Test.txt","r")
gageHeight = open("160921_DailyWaterData_LAbyCounty.txt","r")
csvR = csv.reader(gageHeight, delimiter="\t")
for line in csvR:
    if "#" not in line[0] and "agency_cd" in line[0]:
        siteNo_I = line.index("site_no")
        date_I = line.index("datetime")
        tmp = next((s for s in line if "00003" in s), None)
        #print tmp
        meanH_I = line.index(tmp)
        csvR.next()
    elif "#" not in line[0]:
        gageMeanHeight.append([line[siteNo_I], line[date_I], line[meanH_I]])
#print gageMeanHeight

gageMH = pd.DataFrame(gageMeanHeight,columns=['ID','Date','MeanHeight'])
gageMH.head()




,ID,Date,MeanHeight
0,02489500,1970-10-01,6.57
1,02489500,1970-10-02,6.46
2,02489500,1970-10-03,6.20
3,02489500,1970-10-04,5.97
4,02489500,1970-10-05,5.82


In [6]:
print len(gageMH.index)

1026350


In [7]:
gageMH.Date.iloc[1]

'1970-10-02'

In [8]:
from datetime import datetime
t1 = gageMH.Date.iloc[1]
t1_d = datetime.strptime(t1, '%Y-%M-%d')
print t1_d.year

1970


In [9]:
#convert Date variable from string to datetime:
time = pd.to_datetime(gageMH.Date,infer_datetime_format=True) #(gageMH.Date, '%Y-%M-%d')
gageMH.Date = time
gageMH.Date.iloc[1]

Timestamp('1970-10-02 00:00:00')

In [16]:
la_towns = pd.read_csv("161103_Louisiana_Cities_Parish.csv",header=0) 
la_towns.head()

,Name,Type,Parish,Population (2010),Population (2000),Change,Land area (2010) sq mi,Land area (2010) km2,Density
0,Abbeville,City,Vermilion,"12,257","11,887",+3.1%,6.04,15.6,"2,029.3/sq mi (783.5/km2)"
1,Abita Springs,Town,St. Tammany,"2,365","1,957",+20.8%,4.48,11.6,527.9/sq mi (203.8/km2)
2,Addis,Town,West Baton Rouge,"3,593","2,238",+60.5%,4.20,10.9,855.5/sq mi (330.3/km2)
3,Albany,Village,Livingston,"1,088",865,+25.8%,1.12,2.9,971.4/sq mi (375.1/km2)
4,Alexandria,City,Rapides,"47,723","46,342",+3.0%,28.41,73.6,"1,679.8/sq mi (648.6/km2)"


In [20]:
gageInfo = pd.merge(gageMH,gages,on="ID")
gageInfo.head()


,ID,Date,MeanHeight,City
0,02489500,1970-10-01,6.57,Bogalusa
1,02489500,1970-10-02,6.46,Bogalusa
2,02489500,1970-10-03,6.20,Bogalusa
3,02489500,1970-10-04,5.97,Bogalusa
4,02489500,1970-10-05,5.82,Bogalusa


In [21]:
la_gage_data = pd.merge(gageInfo,la_towns,left_on="City",right_on="Name")
la_gage_data.head()

,ID,Date,MeanHeight,City,Name,Type,Parish,Population (2010),Population (2000),Change,Land area (2010) sq mi,Land area (2010) km2,Density
0,02489500,1970-10-01,6.57,Bogalusa,Bogalusa,City,Washington,"12,232","13,365",?8.5%,9.50,24.6,"1,287.6/sq mi (497.1/km2)"
1,02489500,1970-10-02,6.46,Bogalusa,Bogalusa,City,Washington,"12,232","13,365",?8.5%,9.50,24.6,"1,287.6/sq mi (497.1/km2)"
2,02489500,1970-10-03,6.20,Bogalusa,Bogalusa,City,Washington,"12,232","13,365",?8.5%,9.50,24.6,"1,287.6/sq mi (497.1/km2)"
3,02489500,1970-10-04,5.97,Bogalusa,Bogalusa,City,Washington,"12,232","13,365",?8.5%,9.50,24.6,"1,287.6/sq mi (497.1/km2)"
4,02489500,1970-10-05,5.82,Bogalusa,Bogalusa,City,Washington,"12,232","13,365",?8.5%,9.50,24.6,"1,287.6/sq mi (497.1/km2)"


In [22]:
la_gage_data.drop('Name',axis=1,inplace=True)
la_gage_data.drop('Change',axis=1,inplace=True)
la_gage_data.head()

,ID,Date,MeanHeight,City,Type,Parish,Population (2010),Population (2000),Land area (2010) sq mi,Land area (2010) km2,Density
0,02489500,1970-10-01,6.57,Bogalusa,City,Washington,"12,232","13,365",9.50,24.6,"1,287.6/sq mi (497.1/km2)"
1,02489500,1970-10-02,6.46,Bogalusa,City,Washington,"12,232","13,365",9.50,24.6,"1,287.6/sq mi (497.1/km2)"
2,02489500,1970-10-03,6.20,Bogalusa,City,Washington,"12,232","13,365",9.50,24.6,"1,287.6/sq mi (497.1/km2)"
3,02489500,1970-10-04,5.97,Bogalusa,City,Washington,"12,232","13,365",9.50,24.6,"1,287.6/sq mi (497.1/km2)"
4,02489500,1970-10-05,5.82,Bogalusa,City,Washington,"12,232","13,365",9.50,24.6,"1,287.6/sq mi (497.1/km2)"


In [26]:
a = pd.Series(['ANOTHER TEST','LIVINGSTON'])
a.str.title()

0    Another Test
1      Livingston
dtype: object

In [30]:
la_severeWeather = pd.read_excel("161103_Storm_data_search_results.xlsx")
la_severeWeather['CZ_NAME_STR'] = la_severeWeather['CZ_NAME_STR'].str.title()
la_severeWeather['BEGIN_LOCATION'] = la_severeWeather['BEGIN_LOCATION'].str.title()
la_severeWeather.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,5667808,Acadia,,1998-09-09,300,Tropical Storm,,,0,3,...,,,,,,,65,NaN,NaN,NaN
1,5286230,Acadia,,2002-03-01,1200,High Wind,35,,0,0,...,,,,,,,80,NaN,NaN,NaN
2,5319437,Acadia,,2002-10-03,700,Hurricane (Typhoon),,,0,2,...,,,,,,,85,NaN,NaN,NaN
3,5407647,Acadia,,2004-06-24,1200,Flood,,,0,0,...,,,,,,,94,NaN,NaN,NaN
4,5406313,Acadia,,2004-06-25,1130,Flood,,,0,0,...,,,,,,,95,NaN,NaN,NaN


In [31]:
la_severeWeather.columns.values

array([u'EVENT_ID', u'CZ_NAME_STR', u'BEGIN_LOCATION', u'BEGIN_DATE',
       u'BEGIN_TIME', u'EVENT_TYPE', u'MAGNITUDE', u'TOR_F_SCALE',
       u'DEATHS_DIRECT', u'INJURIES_DIRECT', u'DAMAGE_PROPERTY_NUM',
       u'DAMAGE_CROPS_NUM', u'STATE_ABBR', u'CZ_TIMEZONE',
       u'MAGNITUDE_TYPE', u'EPISODE_ID', u'CZ_TYPE', u'CZ_FIPS', u'WFO',
       u'INJURIES_INDIRECT', u'DEATHS_INDIRECT', u'SOURCE', u'FLOOD_CAUSE',
       u'TOR_LENGTH', u'TOR_WIDTH', u'BEGIN_RANGE', u'BEGIN_AZIMUTH',
       u'END_RANGE', u'END_AZIMUTH', u'END_LOCATION', u'BEGIN_LAT',
       u'BEGIN_LON', u'END_LAT', u'END_LON', u'ABSOLUTE_ROWNUMBER',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37'], dtype=object)

In [42]:
la_sevWe = la_severeWeather.iloc[:,0:12]
la_sevWe.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,DAMAGE_PROPERTY_NUM,DAMAGE_CROPS_NUM
0,5667808,Acadia,,1998-09-09,300,Tropical Storm,,,0,3,5000000,0
1,5286230,Acadia,,2002-03-01,1200,High Wind,35,,0,0,25000,0
2,5319437,Acadia,,2002-10-03,700,Hurricane (Typhoon),,,0,2,50000000,11500000
3,5407647,Acadia,,2004-06-24,1200,Flood,,,0,0,0,0
4,5406313,Acadia,,2004-06-25,1130,Flood,,,0,0,0,0


In [44]:
#df = df[df.line_race != 0]
la_sevWe = la_sevWe.replace('0',np.NaN)
la_sevWe = la_sevWe.dropna(thresh=9)
la_sevWe.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,DAMAGE_PROPERTY_NUM,DAMAGE_CROPS_NUM
0,5667808,Acadia,,1998-09-09,300.0,Tropical Storm,,,NaN,3.0,5000000.0,NaN
1,5286230,Acadia,,2002-03-01,1200.0,High Wind,35,,NaN,NaN,25000.0,NaN
2,5319437,Acadia,,2002-10-03,700.0,Hurricane (Typhoon),,,NaN,2.0,50000000.0,11500000.0
5,5475127,Acadia,,2005-09-23,1200.0,Hurricane (Typhoon),,,NaN,NaN,50000000.0,NaN
6,90238,Acadia,,2008-03-03,1300.0,Strong Wind,43,,NaN,NaN,10000.0,NaN
